In [1]:
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from openpilot.selfdrive.car.ford.values import CAR

database = get_comma_car_segments_database()

platforms = [c.value for c in CAR]

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [2]:
import random

from openpilot.tools.lib.logreader import LogReader

MAX_SEGS_PER_PLATFORM = 10

CAR_FW_TO_CHECK = []
VINS = set()

for platform in platforms:
  if platform not in database:
    print(f'Skipping platform: {platform}, no data available')
    continue
  
  all_segments = database[platform]

  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)

  print(f'Got {len(all_segments)} segments for platform {platform}, sampling {NUM_SEGMENTS} segments')

  segments = random.sample(all_segments, NUM_SEGMENTS)

  for segment in segments:
    lr = LogReader(segment)
    CP = lr.first('carParams')
    if CP.carVin in VINS:
      continue
    VINS.add(CP.carVin)
    CAR_FW_TO_CHECK.append(CP)

Got 287 segments for platform FORD BRONCO SPORT 1ST GEN, sampling 10 segments
Got 137 segments for platform FORD ESCAPE 4TH GEN, sampling 10 segments
Got 1041 segments for platform FORD EXPLORER 6TH GEN, sampling 10 segments
Got 5 segments for platform FORD F-150 14TH GEN, sampling 5 segments
Got 56 segments for platform FORD FOCUS 4TH GEN, sampling 10 segments
Got 637 segments for platform FORD MAVERICK 1ST GEN, sampling 10 segments
Got 3 segments for platform FORD F-150 LIGHTNING 1ST GEN, sampling 3 segments
Got 3 segments for platform FORD MUSTANG MACH-E 1ST GEN, sampling 3 segments


In [7]:
import pandas as pd

from openpilot.selfdrive.car.fw_versions import build_fw_dict, match_fw_to_car

results = []
for CP in CAR_FW_TO_CHECK:
  vin = CP.carVin
  real_fingerprint, real_fuzzy, source = CP.carFingerprint, CP.fuzzyFingerprint, CP.fingerprintSource
  car_fw = CP.carFw

  exact_match, determined_fingerprint = match_fw_to_car(car_fw, log=False)
  determined_fuzzy = not exact_match

  if len(determined_fingerprint) == 1:
    determined_fingerprint = determined_fingerprint.pop()
  elif len(determined_fingerprint) == 0:
    determined_fingerprint = 'mock'
  else:
    determined_fingerprint = 'multiple'

  fw_dict = build_fw_dict(CP.carFw)
  has_radar = (0x764, None) in fw_dict.keys()

  changed = (real_fingerprint != determined_fingerprint) or (real_fuzzy != determined_fuzzy)
  results.append((vin, has_radar,
                  real_fingerprint, real_fuzzy, determined_fingerprint, determined_fuzzy,
                  changed))

  if changed:
    print(f'{vin=}  {real_fingerprint=:<28}  source={source}  {determined_fingerprint=:<28} ({exact_match=})')
    if determined_fingerprint != real_fingerprint:
      for addr, fw in build_fw_dict(car_fw).items():
        print(f'  addr={hex(addr[0])} {fw=}')
    print()

df = pd.DataFrame(results, columns=['vin', 'has_radar', 'real_fingerprint', 'real_fuzzy', 'determined_fingerprint', 'determined_fuzzy', 'changed'])
print(df.to_string())

vin='5LM5J7XC1LGXXXXXX'  real_fingerprint=FORD EXPLORER 6TH GEN         source=fixed  determined_fingerprint=mock                         (exact_match=True)
  addr=0x7e0 fw={b'LB5A-14C204-AZJ\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x730 fw={b'L1MC-14D003-AJ\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x760 fw={b'L1MC-2D053-AJ\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x706 fw={b'LC5T-14F397-AE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x764 fw={b'LB5T-14D049-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}

vin='00000000000XXXXXX'  real_fingerprint=FORD F-150 14TH GEN           source=fixed  determined_fingerprint=mock                         (exact_match=True)

vin='3FTTW8E3XPRXXXXXX'  real_fingerprint=FORD MAVERICK 1ST GEN         source=fw  determined_fingerprint=mock                         (exact_match=True)
  addr=0x7e0 fw={b'PZ6A-14C204-JE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x730 fw={b'NZ6C-14D003-AL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'